In [22]:
'''
Descrição
'''
import argparse, json, logging, time

import apache_beam as beam
import apache_beam.transforms.window as window
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

import time
import datetime

In [279]:
class Checkout(beam.DoFn):
  
  def process(self, element, window=beam.DoFn.WindowParam):
    '''
    Debug
    
    usuario = element[0]
    eventos = str(len(element[1]))
    window_end = window.end.to_utc_datetime()
    window_start = window.start.to_utc_datetime()
    session_duration = window_end - window_start
    print('-----------')
    print('user')
    print(user)
    print('element')
    print(element)
    print('num_events')
    print(num_events)
    print('window_end')
    print(window.max_timestamp().to_utc_datetime())
    print('window_start')
    print(window_start)
    print('session_duration')
    print(session_duration)
    print('chave')
    res = [ sub['page'] for sub in element[1] ]
    print('checkout' not in res)
    print('-----------')
    '''
    pages = [ page['page'] for page in element[1] ]
    if 'checkout' not in pages:
        return element[1]
    

    #yield element

In [290]:
def printer(data_item):
    print(data_item)

In [270]:
def parse_json(data_item):
     yield json.loads(data_item)

In [291]:

p = beam.Pipeline(options=PipelineOptions())

session_gap = 60*10 # 10 minutos

with beam.Pipeline(options=PipelineOptions()) as p:
    carrinho = (p
        | 'Lê Arquivo' >> ReadFromText('input/page-views.json') \
        | 'Parse Json' >> beam.ParDo(parse_json)
        | 'Adiciona Timestamp' >> beam.Map(lambda x: beam.window.TimestampedValue(
            x,datetime.datetime.strptime(x['timestamp'], '%Y-%m-%d %H:%M:%S').timestamp()))
        | 'Adciona chave'      >> beam.Map(lambda x: (x['customer'], x))
        | 'Define Sessão'   >> beam.WindowInto(window.Sessions(session_gap),
                                               timestamp_combiner=window.TimestampCombiner.OUTPUT_AT_EOW)
        | 'Group By Key' >> beam.GroupByKey()
        | 'Analise' >> beam.ParDo(Checkout())
        | 'Imprime Resultado' >> beam.ParDo(printer)
        #| 'Salva arquivo' >> WriteToText('output/teste','.json')
        )

{'timestamp': '2019-01-01 13:00:00', 'customer': 'customer-2', 'product': 'product-2', 'page': 'product'}
{'timestamp': '2019-01-01 13:02:00', 'customer': 'customer-2', 'product': 'product-2', 'page': 'basket'}
